# AUTOMATIC DIFFERENTIATION WITH `TORCH.AUTOGRAD`

在训练神经网络时，最常使用的算法是**反向传播**。在这种算法中，参数（模型权重）根据损失函数相对于给定参数的**梯度**来调整。

为了计算这些梯度，PyTorch有一个内置的求导引擎称为`torch.autograd`。它支持对任何计算图自动计算梯度。

以最简单的一层神经网络为例，输入为`X`，参数为`W`和`b`，和一些损失函数。它可以在PyTorch中以如下方式定义：

In [1]:
import torch

x = torch.ones(5) # 输入张量
y = torch.zeros(3) # 期望输出
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensors, Functions and Computational graph

这段代码定义了如下**计算图**：
![](https://pytorch.org/tutorials/_images/comp-graph.png)
在这个网络中，`w`和`b`是我们需要优化的参数。因此，我们需要能够计算损失函数相对于这些变量的梯度。为了做到这一点，我们设置了这些张量的 `requires_grad` 属性。

> **小贴士**
>
>你可以在创建张量时设置`requires_grad`的值，或者在创建张量之后使用`x.requires_grad_(True)`方法设置。

我们应用于张量来构建计算图的函数实际上是一个`Function`类的对象。这个对象知道如何在*前向*上计算函数，也知道如何在*后向传播*步骤中计算其导数。后向传播函数的引用被存储在张量的`grad_fn`属性中。你可以在[文档](https://pytorch.org/docs/stable/autograd.html#function)中找到更多关于`Function`的信息。

In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f2180037ad0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f2180037a10>


## Computing Gradients

为了优化神经网络中的参数权重，我们需要计算损失函数相对于参数的导数,也就是我们需要在一些固定的`x`和`y`值下计算$ \frac{\partial loss}{\partial w} $和$ \frac{\partial loss}{\partial b} $。为了计算这些导数，我们调用`loss.backward()`，然后从`w.grad`和`b.grad`中检索数值。

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0050, 0.2721, 0.1239],
        [0.0050, 0.2721, 0.1239],
        [0.0050, 0.2721, 0.1239],
        [0.0050, 0.2721, 0.1239],
        [0.0050, 0.2721, 0.1239]])
tensor([0.0050, 0.2721, 0.1239])


> **小贴士**
>
> 我们只能从将`requires_grad`属性设置为`True`的计算图中的叶子节点获取`grad`属性。对于其他所有图中节点，梯度不可用。
>
> 出于性能方面的考虑，我们在一个给定的计算图上只能使用`backward`进行一次梯度计算。如果我们需要在同一个图形上进行多次`backward`调用，我们需要在`backward`调用中传递 `retain_graph=True`。

## Disabling Gradient Tracking

在默认情况下，所有设置`requires_grad=True`的张量的计算过程会被追踪并且支持梯度计算。然而在一些情况下我们并不需要这么做，例如当我们已经训练好了一个模型并且想把它应用于输入数据，也就是说我们只想在网络中做*前向*计算。我们可以使用`torch.no_grad()`块包裹我们的计算代码来停止追踪计算。

In [5]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


另一种达到同样结果的方式是对张量使用`detach()`方法：

In [7]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


这里是一些你想要禁用梯度追踪可能的原因：
- 将你的神经网络中的一些参数标记为**冻结参数**。
- 当只进行前向传播时进行**加速计算**，因为对张量进行计算时禁用梯度追踪会更高效。

## More on Computational Graphs

从概念上讲，autograd在一个由[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)对象组成的有向无环图（DAG）中保存了数据（张量）和所有执行的操作（以及产生的新张量）的记录。在这个DAG中，叶子是输入张量，根是输出张量。通过追踪这个图从根到叶，你可以使用链式规则自动计算梯度。

在前向传播中，autograd同时做了两件事：
- 运行给定的运算来计算一个结果向量。
- 在DAG中保存运算的梯度函数。

当在DAG的根调用`.backward()`时，后向传递就会启动，然后`.autograd`：
- 从每个`.grad_fn`计算梯度，
- 在每个张量的`.grad`属性积累它们
- 使用链式法则，在每条通往叶子张量的路径上传播。

> **小贴士**
>
> **DAGs在PyTorch中是动态的** 需要注意的是，计算图是从头开始重新创建的；在每次调用`.backward()`后，autograd开始填充一个新的图形。这正是允许你在模型中使用控制流语句的原因；如果需要，你可以在每次迭代时改变形状、大小和操作。

## 选读：张量梯度和雅各比（Jacobian）乘积

在许多情况下，我们有一个标量损失函数，我们需要计算相对于某些参数的梯度。然而，有些情况下，输出函数是一个任意的张量。在这种情况下，PyTorch允许你计算所谓的**雅各比乘积**，而不是实际的梯度。

对于一个向量函数$\vec y = f(\vec x)$， 当$\vec x = \langle x_1, \cdots, x_n \rangle$ 且 $\vec y = \langle y_1, \cdots, y_n \rangle$时。